In [ ]:
import numpy as np
import sympy as sp
from IPython.display import Math, display

In [ ]:
def cprint(tex_text):
    display(Math(tex_text))

# Derivatives for the extended GAM implementation

\begin{align*}
y&: y \\
a&: \alpha = \exp(\vartheta_{0}) \\
n&: \eta \\
g&: \gamma \\
\end{align*}
where $\vartheta_{0}\in \mathbb{R}$ is an unrestricted parameter.

In [ ]:
y, t0, n, g = sp.symbols("y, theta0, eta, gamma")
a = sp.exp(t0)

In [ ]:
l_ZTNB = (
    y * sp.log(a)
    + y * g
    - y * sp.log(1 + a * sp.exp(g))
    + sp.log(sp.gamma(y + 1 / a))
    - sp.log(sp.gamma(y + 1))
    - sp.log(sp.gamma(1 / a))
    - sp.log(((1 + a * sp.exp(g)) ** (1 / a)) - 1)
)

cprint(rf"\ell_{{ZTNB}} = {sp.latex(l_ZTNB.subs(a, 'alpha'))}")

In [ ]:
# for y == 0
l0_hurdleNB = -sp.exp(n)
# for y_p = y > 0
l_hurdleNB = sp.log(1 - sp.exp(-sp.exp(n))) + l_ZTNB

cprint(rf"\ell_{{hurdleNB}} = {sp.latex(l0_hurdleNB)}, \quad \text{{for}} ~ y = 0.")
cprint(
    rf"\ell_{{hurdleNB}} = {sp.latex(l_hurdleNB.subs(a, 'alpha'))}, \quad \text{{for}} ~ y > 0."
)

## Derivatives, for $y > 0$

#### w.r.t. $\eta$

In [ ]:
l_e = l_hurdleNB.diff(n, 1)
cprint(rf"\ell_{{\eta}} = {sp.latex(l_e)}")

In [ ]:
l_ee = l_hurdleNB.diff(n, 2).subs(l_e, "l_eta").expand()
cprint(rf"\ell_{{\eta\eta}} = {sp.latex(l_ee)}")

In [ ]:
l_eee = l_hurdleNB.diff(n, 3).subs(l_e, "l_eta").expand()
cprint(rf"\ell_{{\eta\eta\eta}} = {sp.latex(l_eee)}")

In [ ]:
l_eeee = l_hurdleNB.diff(n, 4).subs(l_e, "l_eta").expand()
cprint(rf"\ell_{{\eta\eta\eta\eta}} = {sp.latex(l_eeee)}")

#### w.r.t. $\gamma$

In [ ]:
alpha, kappa, tau = sp.symbols("alpha kappa tau")

k = sp.exp(g) / (1 + alpha * sp.exp(g))
t = ((1 + alpha * sp.exp(g)) ** (1 / alpha)) / (
    (1 + alpha * sp.exp(g)) ** (1 / alpha) - 1
)

In [ ]:
l_g = l_hurdleNB.diff(g, 1).subs(a, alpha).subs(k, kappa).subs(t, tau)
cprint(rf"\ell_{{\gamma}} = {sp.latex(l_g)}")

In [ ]:
l_gg = l_hurdleNB.diff(g, 2).subs(a, alpha).subs(k, kappa).subs(t, tau).expand()

lg = sp.symbols(r"\ell_{\gamma}")
sl_gg = -alpha * kappa * lg - kappa * tau + kappa**2 * tau**2 - kappa**2 * tau
assert sp.simplify(sl_gg.subs(lg, l_g) - l_gg) == 0
cprint(rf"\ell_{{\gamma\gamma}} = {sp.latex(sl_gg)}")

In [ ]:
l_ggg = l_hurdleNB.diff(g, 3).subs(a, alpha).subs(k, kappa).subs(t, tau).expand()

lgg = sp.symbols(r"\ell_{\gamma\gamma}")
sl_ggg = (
    -3 * alpha * kappa * lgg
    + 3 * lgg
    + 2 * alpha * kappa * lg
    - alpha**2 * kappa**2 * lg
    - alpha * kappa**2 * tau
    - 2 * kappa**3 * tau**3
    + 3 * kappa**3 * tau**2
    - kappa**3 * tau
    + 2 * kappa * tau
)
assert sp.simplify(sl_ggg.subs(lg, l_g).subs(lgg, l_gg) - l_ggg) == 0
cprint(rf"\ell_{{\gamma\gamma\gamma}} = {sp.latex(sl_ggg)}")

In [ ]:
# for full-Newton
l_gggg = l_hurdleNB.diff(g, 4).subs(a, alpha).subs(k, kappa).subs(t, tau).expand()

lggg = sp.symbols(r"\ell_{\gamma\gamma\gamma}")
sl_gggg = (
    -3 * alpha * kappa * lggg
    + 6 * lggg
    + 9 * alpha * kappa * lgg
    - 11 * lgg
    - 6 * alpha * kappa * lg
    - 6 * kappa * tau
    + 2 * alpha**2 * kappa**4 * tau**2
    - 2 * alpha**2 * kappa**4 * tau
    + 6 * alpha * kappa**4 * tau**3
    - 9 * alpha * kappa**4 * tau**2
    + 3 * alpha * kappa**4 * tau
    + 6 * kappa**4 * tau**4
    - 12 * kappa**4 * tau**3
    + 7 * kappa**4 * tau**2
    - kappa**4 * tau
)

assert (
    sp.simplify(sl_gggg.subs(lggg, l_ggg).subs(lgg, l_gg).subs(lg, l_g) - l_gggg) == 0
)
cprint(rf"\ell_{{\gamma\gamma\gamma\gamma}} = {sp.latex(sl_gggg)}")

#### w.r.t. $\vartheta_0$

In [ ]:
omega = sp.symbols("omega")
w = (1 / alpha) * sp.log(1 + alpha * sp.exp(g)) - kappa

In [ ]:
l_t0 = l_hurdleNB.diff(t0, 1).subs(a, alpha).subs(k, kappa).subs(t, tau).subs(w, omega)

sl_t0 = (
    lg
    + kappa * tau
    + tau * omega
    + sp.polygamma(0, 1 / alpha) / alpha
    - sp.polygamma(0, y + 1 / alpha) / alpha
)
assert sp.simplify(sl_t0.subs(lg, l_g) - l_t0) == 0
cprint(rf"\ell_{{\vartheta_{0}}} = {sp.latex(sl_t0)}")

In [ ]:
# for full Newton
l_t0t0 = (
    l_hurdleNB.diff(t0, 2).subs(a, alpha).subs(k, kappa).subs(t, tau).subs(w, omega)
)

lt0 = sp.symbols(r"\ell_{\vartheta_{0}}")
sl_t0t0 = (
    -lt0
    - alpha * kappa * lg
    + omega**2 * tau**2
    - omega**2 * tau
    + lg
    + kappa * tau
    - sp.polygamma(1, 1 / alpha) / alpha**2
    + sp.polygamma(1, y + 1 / alpha) / alpha**2
)
assert sp.simplify(sl_t0t0.subs(lg, l_g).subs(lt0, l_t0) - l_t0t0) == 0
cprint(rf"\ell_{{\vartheta_{0}\vartheta_{0}}} = {sp.latex(sl_t0t0)}")

#### Mixed derivatives

Mixed derivatives involving $\eta$ are all 0 in the GAMLSS case. In the Extended GAM case, with $\eta=\vartheta_{1} + \exp(\vartheta_{2})\gamma$, we are interested in mixed derivatives involving $\gamma$ and $\vartheta_{i}, i=0,1,2$.

In [ ]:
l_gt0 = (
    l_hurdleNB.diff(g, 1, t0, 1)
    .subs(a, alpha)
    .subs(k, kappa)
    .subs(t, tau)
    .subs(w, omega)
    .expand()
)

sl_gt0 = -alpha * kappa * lg - kappa * tau**2 * omega + kappa * tau * omega
assert sp.simplify(sl_gt0.subs(lg, l_g) - l_gt0) == 0
cprint(rf"\ell_{{\gamma\vartheta_{0}}} = {sp.latex(sl_gt0)}")

In [ ]:
l_ggt0 = (
    l_hurdleNB.diff(g, 2, t0, 1)
    .subs(a, alpha)
    .subs(k, kappa)
    .subs(t, tau)
    .subs(w, omega)
    .expand()
)

lgt0 = sp.symbols(r"\ell_{\gamma\vartheta_{0}}")
sl_ggt0 = (
    -alpha * kappa * lgt0
    - alpha**2 * kappa**2 * lg
    - 2 * alpha * kappa * lgg
    + lgt0
    - alpha * kappa**2 * tau
    + 2 * kappa**2 * omega * tau**3
    - 3 * kappa**2 * omega * tau**2
    + kappa**2 * omega * tau
)
assert (
    sp.simplify(sl_ggt0.subs(lg, l_g).subs(lgg, l_gg).subs(lgt0, l_gt0) - l_ggt0) == 0
)
cprint(rf"\ell_{{\gamma\gamma\vartheta_{0}}} = {sp.latex(sl_ggt0)}")

In [ ]:
# for full Newton
l_gt0t0 = (
    l_hurdleNB.diff(g, 1, t0, 2)
    .subs(a, alpha)
    .subs(k, kappa)
    .subs(t, tau)
    .subs(w, omega)
    .expand()
)

sl_gt0t0 = (
    -2 * alpha * kappa * lgt0
    - 2 * lg * alpha * kappa
    - lgt0
    - alpha * kappa**3 * tau**2
    + alpha * kappa**3 * tau
    - 2 * kappa * omega**2 * tau**3
    + 3 * kappa * omega**2 * tau**2
    - kappa * omega**2 * tau
)
assert sp.simplify(sl_gt0t0.subs(lg, l_g).subs(lgt0, l_gt0) - l_gt0t0) == 0
cprint(rf"\ell_{{\gamma\vartheta_{0}\vartheta_{0}}} = {sp.latex(sl_gt0t0)}")

In [ ]:
# for full Newton
l_gggt0 = (
    l_hurdleNB.diff(g, 3, t0, 1)
    .subs(a, alpha)
    .subs(k, kappa)
    .subs(t, tau)
    .collect([2, 6, 7, 12])
    .subs(w, omega)
    .expand()
)

lggt0 = sp.symbols(r"\ell_{\gamma\gamma\vartheta_{0}}")
sl_gggt0 = (
    -3 * lggt0 * alpha * kappa
    + 3 * lggt0
    + 3 * lgt0 * alpha * kappa
    - 2 * lgt0
    - 3 * lggg * alpha * kappa
    + 6 * lgg * alpha * kappa
    - 6 * lgg * alpha**2 * kappa**2
    - alpha**2 * kappa**2 * lgt0
    - alpha**3 * kappa**3 * lg
    + 3 * alpha**2 * kappa**2 * lg
    + 3 * alpha * kappa**2 * tau
    - alpha**2 * kappa**3 * tau
    - 6 * kappa**3 * omega * tau**4
    + 12 * kappa**3 * omega * tau**3
    - 7 * kappa**3 * omega * tau**2
    + kappa**3 * omega * tau
)
assert (
    sp.simplify(
        sl_gggt0.subs(lg, l_g)
        .subs(lgg, l_gg)
        .subs(lggg, l_ggg)
        .subs(lgt0, l_gt0)
        .subs(lggt0, l_ggt0)
        - l_gggt0
    )
    == 0
)
cprint(rf"\ell_{{\gamma\gamma\gamma\vartheta_{0}}} = {sp.latex(sl_gggt0)}")

In [ ]:
# for full Newton
l_ggt0t0 = (
    l_hurdleNB.diff(g, 2, t0, 2)
    .subs(a, alpha)
    .subs(k, kappa)
    .subs(t, tau)
    .collect(4)
    .subs(w, omega)
    .expand()
)

lgt0t0 = sp.symbols(r"\ell_{\gamma\vartheta_{0}\vartheta_{0}}")
sl_ggt0t0 = (
    -2 * lggt0 * alpha * kappa
    - lggt0
    - alpha * kappa * lggg
    - lgt0t0 * alpha * kappa
    + lgt0t0
    - lgt0 * alpha * kappa
    + lgt0
    - lgg * alpha**2 * kappa**2
    - alpha * kappa * lgg
    + alpha**3 * kappa**3 * lg
    + alpha**2 * kappa**3 * tau
    + 6 * kappa**2 * omega**2 * tau**4
    - 12 * kappa**2 * omega**2 * tau**3
    + 7 * kappa**2 * omega**2 * tau**2
    - kappa**2 * omega**2 * tau
    - 4 * alpha * kappa**3 * omega * tau**3
    + 6 * alpha * kappa**3 * omega * tau**2
    - 2 * alpha * kappa**3 * omega * tau
)
assert (
    sp.simplify(
        sl_ggt0t0.subs(lg, l_g)
        .subs(lgg, l_gg)
        .subs(lggg, l_ggg)
        .subs(lgt0, l_gt0)
        .subs(lggt0, l_ggt0)
        .subs(lgt0t0, l_gt0t0)
        - l_ggt0t0
    )
    == 0
)
cprint(rf"\ell_{{\gamma\gamma\vartheta_{0}\vartheta_{0}}} = {sp.latex(sl_ggt0t0)}")

Further mixed derivatives are trivial.

Limits w.r.t. $\eta$ are the same as in [Wood et al., 2016, Supplementary Appendix I](https://doi.org/10.1080/01621459.2016.1180986)